In [ ]:
# Using webcam recognize face
import torch
import pyttsx3
import cv2
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1
from torchvision import datasets
from tkinter import *
from PIL import Image, ImageTk
import os
from gtts import gTTS

import numpy as np
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from scipy import stats

import serial
import time
import winsound



In [ ]:
##############

def Vname(namel):
    engine = pyttsx3.init()
    rate = engine.getProperty('rate')   # getting details of current speaking rate
    engine.setProperty('rate', 125)     # setting up new voice rate
    voices = engine.getProperty('voices')       #getting details of current voice
    #engine.setProperty('voice', voices[0].id)  #changing index, changes voices. o for male
    engine.setProperty('voice', voices[1].id)   #changing index, changes voices. 1 for female
    volume = engine.getProperty('volume')   #getting to know current volume level (min=0 and max=1)
    engine.setProperty('volume',1.0)    # setting up volume level  between 0 and 1

    text = "Welcome" + namel
    engine.say(text)
    engine.runAndWait()
    
############### 

# initializing MTCNN and InceptionResnetV1 
mtcnn0 = MTCNN(image_size=240, margin=0, keep_all=False, min_face_size=40) # keep_all=False if multiple faces keep all false and detect one only
mtcnn = MTCNN(image_size=240, margin=0, keep_all=True, min_face_size=40) # keep_all=True to keep all faces in form of list
resnet = InceptionResnetV1(pretrained='vggface2').eval()  # using vggface2 pretrained model 

# # Read data from folder

# loading data.pt file
load_data = torch.load('data.pt') 
embedding_list = load_data[0] 
name_list = load_data[1] 



def face():
    current_name = None
    counter_dakhel = 0
    counter_yaman = 0
    counter_mohammed = 0
    namev = 0 
    n = 8
    namel = ""


    while True:
        
        ret, frame = vid.read()
        if not ret:
            print("fail to grab frame, try again")
            break

        img = Image.fromarray(frame)
        img_cropped_list, prob_list = mtcnn(img, return_prob=True) 

        if img_cropped_list is not None:
            boxes, _ = mtcnn.detect(img)

            for i, prob in enumerate(prob_list):
                if prob>0.90:
                    emb = resnet(img_cropped_list[i].unsqueeze(0)).detach() 

                    dist_list = [] # list of matched distances, minimum distance is used to identify the person

                    for idx, emb_db in enumerate(embedding_list):
                        dist = torch.dist(emb, emb_db).item()
                        dist_list.append(dist)

                    min_dist = min(dist_list) # get minumum dist value
                    min_dist_idx = dist_list.index(min_dist) # get minumum dist index
                    name = name_list[min_dist_idx] # get name corrosponding to minimum dist

                    box = boxes[i] 

                    original_frame = frame.copy() # storing copy of frame before drawing on it


                    if min_dist < 0.90:
                        namel = name
                        if current_name != namel:
                            current_name = namel
                            counter_dakhel = 0
                            counter_yaman = 0
                            counter_mohammed = 0
                        if namel == "Dakhel":
                            counter_dakhel += 1
                            print(f"Name: {namel} - Count: {counter_dakhel}")
                        elif namel == "Yaman":
                            counter_yaman += 1
                            print(f"Name: {namel} - Count: {counter_yaman}")
                        elif namel == "Mohammed":
                            counter_mohammed += 1
                            print(f"Name: {namel} - Count: {counter_mohammed}")



#         cv2.imshow("IMG", frame)


        k = cv2.waitKey(1)
        if k%256==27: # ESC
            print('Esc pressed, closing...')
            break
        
        xp = 330
        yp = 225
        
        if counter_dakhel == n:
            namev=1        
            print("Welcome :" + namel + "- Name Index : " + str(namev))
            Vname(namel)
            break
        
        if counter_mohammed == n:
            namev=2
            print("Welcome :" + namel + "- Name Index : " + str(namev)) 
            Vname(namel)
            break

        if counter_yaman == n:
            namev=3
            print("Welcome :" + namel + " - Name Index : " + str(namev)) 
            Vname(namel)
            break
#     vid.release()
#     cv2.destroyAllWindows()
    return namel





In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(120,120,120), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(1,1,1), thickness=2, circle_radius=2))
def extract_keypoints(results):
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([rh])

##############

# Actions that we try to detect
actions = np.array(['zero', 'one', 'two', 'three', 'four', 'five'])

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

model = Sequential()
model.add(LSTM(32, return_sequences=True, activation='relu', input_shape=(30,63)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.load_weights('actionDetection.h5')

colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame



In [ ]:
def signM():

    # Text to Speech Module
    engine = pyttsx3.init()
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[1].id)
    volume = engine.getProperty('volume')
    engine.setProperty('volume',1.0)
    rate = engine.getProperty('rate')
    engine.setProperty('rate', 125) 
    
    # 1. New detection variables
    sequence = []
    sentence = []
    predictions = []
    threshold = 0.7

#     vid = cv2.VideoCapture(0)

    # Set mediapipe model 
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while vid.isOpened():

            # Read feed
            ret, frame = vid.read()

            # Make detections
            global image1
            
            image, results = mediapipe_detection(frame, holistic)

            # Draw landmarks
            draw_landmarks(image, results)

            # 2. Prediction logic
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-30:]

            if len(sequence) == 30:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                predictions.append(np.argmax(res))

            #3. Viz logic
                if np.unique(predictions[-10:])[0]==np.argmax(res): 
                    if res[np.argmax(res)] > threshold: 

                        if len(sentence) > 0: 
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])

                if len(sentence) > 10: 
                    sentence = sentence[-10:]

            cv2.rectangle(image, (0,0), (640, 40), (0, 0, 0), -1)
            cv2.putText(image, ' '.join(sentence), (3,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Authentication
            if len(sentence) == 4:
                time.sleep(2)

                break
                
            image1= image
            
            # Show to screen
            cv2.imshow('OpenCV Feed', image1)


            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        
#         vid.release()
        cv2.destroyAllWindows()
        
    return sentence

In [ ]:
# Define a video capture object
# Create a GUI app

tColor = "black"
pFont = "Cascadia Code"
app = Tk()

app.geometry("666x620")

# Bind the app with Escape keyboard to
# quit app whenever pressed
app.bind('<Escape>', lambda e: app.quit())

# Create a label and display it on app
label_widget = Label(app)
# app["bg"] = "#252526"

label_widget.pack()
app.title("Authentication System")

###################

engine = pyttsx3.init()
rate = engine.getProperty('rate')   # getting details of current speaking rate
engine.setProperty('rate', 125)     # setting up new voice rate
voices = engine.getProperty('voices')       #getting details of current voice
#engine.setProperty('voice', voices[0].id)  #changing index, changes voices. o for male
engine.setProperty('voice', voices[1].id)   #changing index, changes voices. 1 for female
volume = engine.getProperty('volume')   #getting to know current volume level (min=0 and max=1)
engine.setProperty('volume',1.0)    # setting up volume level  between 0 and 1

ser = serial.Serial('COM16', 9600)  # Replace 'COM3' with the serial port used by your Arduino

def arduino():
    ser.write(b'H')
    ser.close()
    time.sleep(5)
    ser.open()


global vid
vid = cv2.VideoCapture(0)

# Declare the width and height in variables
width, height = 600, 600

# Set the width and height
vid.set(cv2.CAP_PROP_FRAME_WIDTH, width)
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

# Create a canvas to display the camera feed
global canvas
canvas = Canvas(app, width=850, height=600)
canvas.place(x=10, y=60)

namex = 10
namey = 10
text_ni = Label(app, text="Name :" + " "  ,fg=tColor, font=(pFont, 10))
text_ni.place(x=namex,y=namey)

passx = 10
passy = 30
text_pi = Label(app, text="Entered Passwored :" + " "  ,fg=tColor, font=(pFont, 10))
text_pi.place(x=passx,y=passy)


line_pi = Label(app, text="|"  ,fg="gray", font=(pFont, 30))
line_pi.place(x=410,y=2)

statusx = 435
statusy = 10
status_pi = Label(app, text="Status : "  ,fg=tColor, font=(pFont, 10))
status_pi.place(x=statusx,y=statusy)


def Access_Granted():
    status_p1 = Label(app, text="Status : " + "Access Granted" ,fg="green", font=(pFont, 10))
    status_p1.place(x=statusx,y=statusy)
    engine.say("Access Granted , Door opening ")
    engine.runAndWait()
    engine.stop()
    winsound.Beep(2637,600)

    ser.write(b'H')
    ser.close()
    time.sleep(6)
    ser.open()

    
def Access_Denied():
    status_p2 = Label(app, text="Status : " + "Access Denied" ,fg="red", font=(pFont, 10))
    status_p2.place(x=statusx,y=statusy)
    
    engine.say("Access Denied, please try again")
    engine.runAndWait()
    engine.stop()


def action1():
    global namef
    namef = face()
    
    text_n = Label(app, text="Name :" + " "+ namef ,fg=tColor, font=(pFont, 10))
    text_n.place(x=namex,y=namey)

    #     text_w = Label(app, text="Welcome" + " " +namef,fg=tColor, font=(pFont, 20))
    #     text_w.place(x=x_text_w,y=y_text_w)


def action2():

    global takenpass
    takenpass = signM()
    takenpassT = "[ " + takenpass[0] + " , " + takenpass[1] + " , " + takenpass[2] + " , " + takenpass[3] + " ]" 
    
    text_p2 = Label(app, text="                                            "  ,fg=tColor, font=(pFont, 10))
    text_p2.place(x=passx,y=passy)
    
    text_p = Label(app, text="Entered Passwored :" + " " + takenpassT ,fg=tColor, font=(pFont, 10))
    text_p.place(x=passx,y=passy)
    
    if namef == "Dakhel" and takenpass ==  ['one', 'four', 'two', 'one']:
        Access_Granted()     
    elif namef == "Yaman" and takenpass ==  ['five', 'three', 'one', 'zero']:
        Access_Granted()
    elif namef == "Mohammed" and takenpass ==  ['four', 'zero', 'one', 'five']:
        Access_Granted()
    else :
        Access_Denied()


# Create a button to open the camera in GUI app
button1 = Button(app, text="Face Detection" , fg=tColor, font=(pFont, 20), activebackground="light blue", height=1, width=19, command=action1)
button1.place(x=10,y=550)

button2 = Button(app, text="Sign Password" , fg=tColor, font=(pFont, 20), activebackground="light blue", height=1, width=19, command=action2)
button2.place(x=340,y=550)


# Start the camera feed
while True:
    global ret
    global frame
    
    ret, frame = vid.read()
    if ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(frame)
        image = ImageTk.PhotoImage(image)
        canvas.create_image(0, 0, image=image, anchor=NW)
    app.update()
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
               
text_w = Label(app, text="Welcome" + " " +namef,fg=tColor, font=(pFont, 20))
text_w.place(x=x_text_w,y=y_text_w)

vid.release()
cv2.destroyAllWindows()

app.mainloop()


In [ ]:
# import serial
# import time


# ser = serial.Serial('COM16', 9600)  # Replace 'COM3' with the serial port used by your Arduino


# # def arduino():
# #     ser.write(b'H')
# #     ser.close()
# # #     time.sleep(5)
# #     ser.open()



In [ ]:
# ser.write(b'H')
# ser.close()
# time.sleep(3)
# ser.open()